# Computer Vision Project - Classification of Flowers


In this project your objective is to create a model in order to classify flowers. Thiszip file contains all relevant data. 

1. The data contains two folders: *train* and *test*. The *train* folder consists of 5486-images to use for training while the *test* folder contains 1351-images you can use to test your model in a **train-test-split** validation style. We have omitted another set of 1352 validation images which we will use to benchmark your final models in the last lecture. 


2. We have provided you with two label files: *train_labels.csv* and *test_labels.csv*. Each file contains the filename of the corresponding image and the class label. In total we have **102 different classes** of flowers.  You can import the label files using the `import_labels()` function provided to you in this notebook.


3. Due to the large number of images, there is a good chance that you can not easily fit the entire training and testing data into RAM. We therefore give you an implementation of a `DataGenerator` class that can be used with keras. This class will read in the images from your hard-drive for each batch during during or testing. The class comes with some nice features that could improve your training significantly such as **image resizing**, **data augmentation** and **preprocessing**. Have a look at the code to find out how.

    Initialize data generators using labels and image source directory.

    `
    datagen_train = DataGenerator('train', y_train, batch_size, input_shape, ...)
    datagen_test = DataGenerator('test', y_test, batch_size, input_shape, ...)`

    Train your model using data generators.

    `model.fit(datagen_train, validation_data=datagen_test, ...)`
    
    
4. Select a suitable model for classification. It is up to you to decide all model parameters, such as **number of layers**, **number and size of filter** in each layer, using **pooling** or, **image-size**, **data-augmentation**, **learning rate**, ... 


5. **Document** your progress and your intermediate results (your failures and improvements). Describe why you selected certain model and training parameters, what worked, what did not work. Store the training history (loss and accuracy) and create corresponding plots. This documentation will be part of your final presentation and will be **graded**.


6. Feel free to explore the internet for suitable CNN models and re-use these ideas. If you use certain features we have not touched during the lecture such as Dropout, Residual Learning or Batch Normalization. Prepare a slide in your final presentation to explain in your own (basic) terms what these things to so we can all learn from your experience. **Notice:** Very large models might perform better but will be harder and slower to train. **Do not use a pre-trained model you find online!**


7. Prepare a notebook with your model such that we can use it in the final competition. This means, store your trained model using `model.save(...)`. Your saved models can be loaded via `tf.keras.models.load_model(...)`. We will then provide you with a new folder containing images (*validation*) and a file containing labels (*validation_labels.csv*) which have the same structure. Prepare a data generator for this validation data (test it using the test data) and supply it to the 
 `evaluate_model(model, datagen)` function provided to you.
 
 Your prepared notebook could look like this:
 
    `... import stuff 
    ... code to load the stored model ...
    y_validation = import_labels('validation_labels.csv')
    datagen_validation = DataGenerator('validation', y_validation, batch_size, input_shape)
    evaluate_model(model, datagen_validation)`


8. Prepare a 15-Minute presentation of your findings and final model presentation. A rough guideline what could be interesting to your audience:
    * Explain your models architecture (number of layers, number of total parameters, how long took it to train, ...)
    * Compare the training history of your experimentats visually
    * Explain your best model (why is it better)
    * Why did you take certain decision (parameters, image size, batch size, ...)
    * What worked, what did not work (any ideas why?)
    * **What did you learn?**
    



In [3]:
# Read in label file and return a dictionary {'filename' : label}.
#
def import_labels(label_file):
    labels = dict()

    import csv
    with open(label_file) as fd:
        csvreader = csv.DictReader(fd)

        for row in csvreader:
            labels[row['filename']] = int(row['label'])
    return labels

In [5]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import image

import numpy as np
import tensorflow as tf

class DataGenerator(keras.utils.Sequence):
    def __init__(self, img_root_dir, labels_dict, batch_size, target_dim, preprocess_func=None, use_augmentation=False):
        self._labels_dict = labels_dict
        self._img_root_dir = img_root_dir
        self._batch_size = batch_size
        self._target_dim = target_dim
        self._preprocess_func = preprocess_func
        self._n_classes = len(set(self._labels_dict.values()))
        self._fnames_all = list(self._labels_dict.keys())
        self._use_augmentation = use_augmentation

        if self._use_augmentation:
            self._augmentor = image.ImageDataGenerator(
                rotation_range=50,
                width_shift_range=0.3,
                height_shift_range=0.3,
                shear_range=0.3,
                zoom_range=0.3,
                horizontal_flip=True,
                fill_mode='nearest'
            )
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self._fnames_all)) / self._batch_size)

    def on_epoch_end(self):
        self._indices = np.arange(len(self._fnames_all))
        np.random.shuffle(self._indices)

    def __getitem__(self, index):
        indices = self._indices[index * self._batch_size:(index+1)*self._batch_size]

        fnames = [self._fnames_all[k] for k in indices]
        X,Y = self.__load_files__(fnames)

        return X,Y

    def __load_files__(self, batch_filenames):
        X = np.empty((self._batch_size, *self._target_dim, 3))
        Y = np.empty((self._batch_size), dtype=int)

        for idx, fname in enumerate(batch_filenames):
            img_path = os.path.join(self._img_root_dir, fname)
            img = image.load_img(img_path, target_size=self._target_dim)
            x = image.img_to_array(img)
            if self._preprocess_func is not None:
                x = self._preprocess_func(x)

            X[idx,:] = x 
            Y[idx] = self._labels_dict[fname]-1

        if self._use_augmentation:
            it = self._augmentor.flow(X, batch_size=self._batch_size, shuffle=False)
            X = it.next()

        if self._preprocess_func is not None:
            X = self._preprocess_func(X)
        return X, tf.keras.utils.to_categorical(Y, num_classes=self._n_classes)

In [7]:
from tensorflow.keras.utils import to_categorical
y_train = import_labels("train_labels.csv")
y_test = import_labels("test_labels.csv")
y_val = import_labels("validation_labels.csv")
batch_size= 24
image_size = (224, 224)

self._augmentor = image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [8]:
def preprocess_func(x):
    x = x/255.0
    return x

In [9]:
datagen_train = DataGenerator('train', y_train, batch_size, image_size, preprocess_func=preprocess_func, use_augmentation=True)
datagen_test = DataGenerator('test', y_test, 8, image_size, preprocess_func=preprocess_func)
datagen_val = DataGenerator("validation", y_val, 8, image_size, preprocess_func=preprocess_func)

In [14]:
def FCT(i,cout):
    dconv = layers.Conv2D(i.shape[3], (4,4),strides=(2,2),groups=i.shape[3], padding="same", use_bias=False)(i)
    max = layers.MaxPooling2D(2)(i)
    min = -layers.MaxPooling2D(2)(-i)
    conc = layers.Concatenate()([max,min, dconv])
    pconv = layers.Conv2D(cout,(1,1), padding="same", use_bias=False)(conc)
    return layers.BatchNormalization()(pconv)

In [15]:
def EVE(i,cout):
    max = layers.MaxPooling2D(2)(i)
    min = -layers.MaxPooling2D(2)(-i)
    conc = layers.Concatenate()([max,min])
    pconv = layers.Conv2D(cout,(1,1), padding="same", use_bias=False)(conc)
    return layers.BatchNormalization()(pconv)

def ME(i,cout):
    max = layers.MaxPooling2D(2)(i)
    pconv = layers.Conv2D(cout,(1,1), padding="same", use_bias=False)(max)
    return layers.BatchNormalization()(pconv)

In [16]:
def SELN(i):
    gavg = layers.GlobalAveragePooling2D()(i)
    norm = layers.LayerNormalization()(gavg)
    act = layers.Activation("sigmoid")(norm)
    mul = layers.multiply([act, i])
    return mul

def SE(i,ratio):
    gavg = layers.GlobalAveragePooling2D()(i)
    fc1 = layers.Dense(int(i.shape[3]/ratio), use_bias=False, activation="relu")(gavg)
    fc2 = layers.Dense(i.shape[3], use_bias=False, activation="sigmoid")(fc1)
    mul = layers.multiply([fc2, i])
    return mul

def DW(i, dw_s):
    dw = layers.Conv2D(i.shape[3], (dw_s,dw_s), groups=i.shape[3], padding="same")(i)
    act1 = layers.Activation("swish")(dw)
    return act1

In [17]:
def DFSEBV2(i,dw_s):
    pconv1 = layers.Conv2D(i.shape[3],(1,1), padding="same", use_bias=False)(i)
    bnorm1 = layers.BatchNormalization()(pconv1)
    act1 = layers.Activation("swish")(bnorm1)
    dconv1 = layers.Conv2D(i.shape[3],(dw_s,dw_s),groups=i.shape[3], padding="same")(act1)
    seln1 = SE(dconv1,3)
    #seln1 = SELN(dconv1)

    add1 = layers.add([seln1, i])
    
    pconv2 = layers.Conv2D(i.shape[3],(1,1),padding="same", use_bias=False)(add1)
    bnorm2 = layers.BatchNormalization()(pconv2)
    act2 = layers.Activation("swish")(bnorm2)
    dconv2 = layers.Conv2D(i.shape[3],(dw_s,dw_s),groups=i.shape[3], padding="same")(act2)
    add2 = layers.add([dconv2, i])
    return add2

In [18]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import layers, losses
#https://arxiv.org/ftp/arxiv/papers/2105/2105.09008.pdf
def get_model():
    num_filters=96
    inputs = layers.Input(image_size + (3,))
    inputs = layers.BatchNormalization()(inputs)
    t = FCT(inputs,num_filters)
    t = DFSEBV2(t,3)
    t = EVE(t,num_filters*4)
    t = DFSEBV2(t,3)
    for i in range(1,4):
        t = ME(t,(num_filters*4)*2**i)
        t = DFSEBV2(t,3)

    t = DW(t, 3)
    t = layers.GlobalAveragePooling2D()(t)
    t = layers.Dropout(.5)(t)
    outputs = layers.Dense(102, activation="softmax", dtype="float32")(t)    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.01), loss=losses.CategoricalCrossentropy(from_logits=False), metrics=["accuracy"])
    model.summary()
    return model

#tuner = kt.RandomSearch(
#    get_model,
#    objective='val_loss',
#    max_trials=5)
model = get_model()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 tf.math.negative (TFOpLambda)  (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_1 (MaxPooling2D)  (None, 112, 112, 3)  0          ['tf.math.negative[1][0]']       
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 112, 112, 3)  0           ['input_2[0][0]']            

In [19]:
import tensorflow as tf
rlr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="loss",
    factor=0.1,
    patience=5,
)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True)

In [21]:
# Train model on dataset
history = model.fit(datagen_train, validation_data=datagen_test, epochs=120,callbacks=[early_stopping, rlr])

In [22]:
import matplotlib.pyplot as plt
try:
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.savefig("06_06_v12_acc.png")
    # summarize history for loss

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.savefig("06_06_v12_loss.png")
except:
    pass

In [23]:
results = model.evaluate(datagen_val)
print("test loss, test acc:", results)


168/168 [==============================] - 144s 822ms/step - loss: 0.4633 - accuracy: 0.9301
test loss, test acc: [0.46330341696739197, 0.930059552192688]


##### 